<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/11_MTEncoderDecoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a usar redes encoder-decoder para hacer traducción automática (MT)

-----------------------

Tarea: revisar todo el código y responder donde dice **PREGUNTA**

### Configuración del entorno


In [ ]:
!pip install -qU transformers accelerate datasets sacrebleu evaluate sacremoses watermark

In [ ]:
%reload_ext watermark

In [ ]:
%watermark -vmp sacrebleu,sacremoses,transformers,datasets,torch,numpy,pandas,tqdm

Python implementation: CPython
Python version       : 3.11.12
IPython version      : 7.34.0

sacrebleu   : 2.5.1
sacremoses  : 0.1.1
transformers: 4.52.3
datasets    : 3.6.0
torch       : 2.6.0+cu124
numpy       : 2.0.2
pandas      : 2.2.2
tqdm        : 4.67.1

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.123+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



Para usar GPU, arriba a la derecha seleccionar "Change runtime type" --> "T4 GPU"

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Encoder-decoder 101

Inicializamos un encoder-decoder aleatoriamente a partir de un encoder y un decoder que tienen la misma estructura.

In [ ]:
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel

config_encoder = BertConfig()
config_decoder = BertConfig()

config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
model = EncoderDecoderModel(config=config)

In [ ]:
model

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

También podemos inicializar a partir de encoders y/o decoders pre-entrenados.

OJO: dependiendo del decoder que usemos, hay capas que pueden venir pre-entrenadas o no.

In [ ]:
from transformers import EncoderDecoderModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = EncoderDecoderModel.from_encoder_decoder_pretrained("google-bert/bert-base-uncased", "google-bert/bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertLMHeadModel were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.wei

**PREGUNTA**: ¿qué capas no están preentradas en el modelo anterior? ¿por qué?

Si cargamos un checkpoint de tipo `EncoderDecoderModel` que ya fue entrenado para una tarea, podemos hacer inferencia directamente. Acá cargamos uno que fue fine-tuneado para resumen de texto.

In [ ]:
from transformers import AutoTokenizer, EncoderDecoderModel

model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")
tokenizer = AutoTokenizer.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")

ARTICLE_TO_SUMMARIZE = (
    "Then came the crucial play of the night. The referee first called six minutes of added time."
    " During that time, there was a long medical attention to Juan Pablo Cozzani, Platense's goalkeeper."
    " Correctly, Falcón Pérez decided to add another three minutes. At that moment, after a quick side"
    " kick, there was a frontal ball, Lanzini headed and the ball was left to Borja, who entered the"
    " area and was brought down by Salomón. Falcón Pérez did not hesitate and gave River a penalty in"
    " what was almost the last play of the night, because eight minutes of added time were already played."
)
input_ids = tokenizer(ARTICLE_TO_SUMMARIZE, return_tensors="pt").input_ids

# Generacion autorregresiva
model.eval()
with torch.inference_mode():
    generated_ids = model.generate(input_ids, do_sample=True, temperature=0.3)
    generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
print(generated_ids)

tensor([[  101,  1996,  5330,  2034,  2170,  2416,  2781,  1997,  2794,  2051,
          1012,  5348, 11623,  2522, 20715,  3490,  2001,  2716,  2091,  2011,
         16183, 19506,  2078,  1012,  1996,  5330,  2059,  2435,  2314,  1037,
          6531,  1999,  2054,  2001,  1996,  2197,  2377,  1997,  1996,  2305,
          1012, 11562,  2182,  2005,  2035,  1996,  6745,  2013,  1996,  2305,
          1005,  1055,  2377,  1011,  2125,  2444,  1012,   102]])


In [ ]:
import textwrap

print(textwrap.fill(generated_text, 100))

the referee first called six minutes of added time. juan pablo cozzani was brought down by salomon.
the referee then gave river a penalty in what was the last play of the night. click here for all the
latest from the night's play - off live.


In [ ]:
# Con beam search:
generated_ids = model.generate(input_ids, do_sample=False, num_beams=4, num_return_sequences=4)
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)

In [ ]:
print(len(generated_ids), len(generated_text))

4 4


In [ ]:
for text in generated_text:
    print(textwrap.fill(text, 100))
    print()

[CLS] the referee first called six minutes of added time. juan pablo cozzani was brought down by
salomon. the referee then gave river a penalty in what was the last play of the night. click here
for all the latest from the night's play - off live. [SEP]

[CLS] the referee first called six minutes of added time. juan pablo cozzani was brought down by
salomon. the referee then gave river a penalty in what was the last play of the night. click here
for all the latest from the night's play - offs. [SEP] [SEP]

[CLS] the referee first called six minutes of added time. juan pablo cozzani was brought down by
salomon. the referee then gave river a penalty in what was the last play of the night. click here
for all the latest from the night's play - off. [SEP] [SEP]

[CLS] the referee first called six minutes of added time. juan pablo cozzani was brought down by
salomon. the referee then gave river a penalty in what was the last play of the night. click here
for all the latest from the night's p

**PREGUNTA**: ¿por qué podemos obtener muchas generaciones con beam search?

Ver más sobre generacion en: https://huggingface.co/docs/transformers/main/en/generation_strategies

In [ ]:
# Esto nos confirma que el decoder está haciendo CLM informado por el decoder
print(model.config.decoder.is_decoder)
print(model.config.decoder.add_cross_attention)

True
True


In [ ]:
# Tambien podemos revisar las matrices de atencion
text = "This is a test"
input_ids = tokenizer(text, return_tensors="pt", padding=True, truncation=True).input_ids

with torch.inference_mode():
    outputs = model.generate(
        input_ids,
        output_attentions=True,
        return_dict_in_generate=True,
        do_sample=False,
        num_beams=1,
    )
generated_text = tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)

The following generation flags are not valid and may be ignored: ['early_stopping', 'length_penalty']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


**PREGUNTA**: ¿qué estrategia de generación de texto acabamos de usar?

In [ ]:
outputs.keys()

odict_keys(['sequences', 'encoder_attentions', 'decoder_attentions', 'cross_attentions', 'past_key_values'])

In [ ]:
print(len(input_ids[0])) # tokens de input (source)
print(len(outputs.sequences[0])) # tokens generados + el de start
print(len(outputs.decoder_attentions)) # cantidad de pasos de generacion
print(len(outputs.cross_attentions))

6
60
59
59


In [ ]:
# Para cada paso de generacion podemos analizar la matriz de atencion en cada capa
# e.g. a qué le prestamos atención en el paso nro 20, capa 8?

print(len(outputs.cross_attentions[20])) # nro de capas
print(outputs.cross_attentions[20][8].shape) # generaciones (>1 en beam search), heads, step actual, source tokens
print(outputs.decoder_attentions[20][8].shape) # generaciones, heads, step actual, previous target tokens

12
torch.Size([1, 12, 1, 6])
torch.Size([1, 12, 1, 21])


**PREGUNTA**: ¿qué representa el siguiente tensor?

In [ ]:
print(outputs.cross_attentions[20][8][0][4][0])

tensor([0.2351, 0.0869, 0.0769, 0.0707, 0.0505, 0.4799])


Para ajustar el modelo a una tarea particular necesitamos:

* elegir un checkpoint pre-entrenado para usar como inicialización
* datos de entrenamiento _(source, target)_
* una función de pérdida

En este caso inicializamos el encoder y decoder por separado pero podríamos usar un encoder-decoder completo pre-entrenado e.g. T5.

In [ ]:
from transformers import AutoTokenizer, EncoderDecoderModel

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
model = EncoderDecoderModel.from_encoder_decoder_pretrained("FacebookAI/xlm-roberta-base", "FacebookAI/xlm-roberta-base")

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

input_ids = tokenizer(
    "Volare nel blu dipinto di blu",
    return_tensors="pt",
).input_ids

labels = tokenizer(
    "volar en el azul pintado de azul",
    return_tensors="pt",
).input_ids

Some weights of XLMRobertaForCausalLM were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['roberta.encoder.layer.0.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.0.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.0.crossattention.output.dense.bias', 'roberta.encoder.layer.0.crossattention.output.dense.weight', 'roberta.encoder.layer.0.crossattention.self.key.bias', 'roberta.encoder.layer.0.crossattention.self.key.weight', 'roberta.encoder.layer.0.crossattention.self.query.bias', 'roberta.encoder.layer.0.crossattention.self.query.weight', 'roberta.encoder.layer.0.crossattention.self.value.bias', 'roberta.encoder.layer.0.crossattention.self.value.weight', 'roberta.encoder.layer.1.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.1.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.1.crossattention.output.dense.bias', 'roberta.encoder.layer.1.crossattention.output.dense.weight', 'r

In [ ]:
# Esto ya calcula la pérdida sobre los tokens correctos
model.train()
loss = model(input_ids=input_ids, labels=labels).loss
print(loss)

/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:557: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)


tensor(16.3715, grad_fn=<NllLossBackward0>)


/usr/local/lib/python3.11/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:577: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


**PREGUNTA**: ¿podemos usar el modelo de arriba directamente para hacer inferencia? ¿por qué usaríamos XLM-R en lugar de bert-base?

## Machine Translation

Queremos hacer fine-tuning de un modelo encoder-decoder ya existente -- puede ser uno que solo fue pre-entrenado o que también haya sido previamente fine-tuneado para MT.

En este caso vamos a ajustar el modelo [MarianMT](https://huggingface.co/docs/transformers/en/model_doc/marian#implementation-notes) (encoder-decoder con 6 capas en cada componente) en el dataset EuroParl, que tiene oraciones alineadas de debates en el parlamento de la UE. EuroParl es parte de OPUS, el dataset de entrenamiento de MarianMT (también llamados modelos Opus de Helsinki).


### Dataset

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("Helsinki-NLP/europarl", "es-it") # de español a italiano

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1880982
    })
})

In [ ]:
# armamos un set de validación:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=33)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1692883
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 188099
    })
})

In [ ]:
# renombramos a "validation":
split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
# Nos quedamos con menos ejemplos para correr más rápido:
split_datasets["train"] = split_datasets["train"].shuffle(seed=33).select(range(2_000))
split_datasets["validation"] = split_datasets["validation"].shuffle(seed=33).select(range(2_000))

In [ ]:
# veamos un ejemplo
split_datasets["train"][33]["translation"]

{'es': 'Eso no nos ayudará a avanzar.',
 'it': "E' un atteggiamento che non è di alcun aiuto."}

Como siempre, necesitamos tokenizar. Muchas veces conviene hacer esto durante el loop de entrenamiento e.g. si la tokenización demora mucho (para experimentar más rápido) o si tenemos un dataset muy grande que no entra en memoria. En este caso vamos a tokenizar todo al principio.

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-es-it" # modelo ya finetuneado en español-italiano

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [ ]:
# Veamos un ejemplo:
source_sentence = split_datasets["train"][33]["translation"]["es"]
target_sentence = split_datasets["train"][33]["translation"]["it"]

inputs = tokenizer(source_sentence, text_target=target_sentence)
inputs

{'input_ids': [731, 31, 307, 9895, 7, 11165, 2, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'labels': [57, 8, 16, 10746, 19, 38, 42, 6, 2648, 1596, 2, 0]}

In [ ]:
# el tokenizador ya nos devuelve source y target tokenizados
# OJO: si no indicamos que estamos tokenizando las etiquetas, lo va a hacer el tokenizador
# source, que no es correcto para este modelo que tiene tokenizadores distintos para source y target:
wrong_targets = tokenizer(target_sentence)

print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁E', "'", '▁un', '▁a', 'tte', 'g', 'gia', 'mento', '▁che', '▁no', 'n', '▁', 'è', '▁di', '▁al', 'cu', 'n', '▁a', 'i', 'uto', '.', '</s>']
['▁E', "'", '▁un', '▁atteggiamento', '▁che', '▁non', '▁è', '▁di', '▁alcun', '▁aiuto', '.', '</s>']


**PREGUNTA**: ¿qué diferencia hay en las tokenizaciones de la celda anterior?

In [ ]:
# Función de tokenizacion
# * Usamos un max_len chico porque asumimos que tenemos oraciones cortas -- OJO hay que verificar esto en la practica!
# * Muchos modelos (como T5) esperan que las entradas tengan un prefijo que indique
#   la tarea en cuestión, e.g. `translate: Spanish to Italian:`

max_length = 128

def preprocess_function(examples):
    inputs = [ex["es"] for ex in examples["translation"]]
    targets = [ex["it"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [ ]:
# veamos los tamaños
for i in range(5):
    doc = tokenized_datasets["train"][i]
    for k, v in doc.items():
        print(f"{k}: {len(v):3}", end=", ")
    print()

input_ids:  52, attention_mask:  52, labels:  46, 
input_ids:  47, attention_mask:  47, labels:  49, 
input_ids:  17, attention_mask:  17, labels:  13, 
input_ids:  16, attention_mask:  16, labels:  17, 
input_ids:   5, attention_mask:   5, labels:   5, 


### Métrica de evaluación

Es FUNDAMENTAL saber cómo vamos a evaluar antes de ponernos a entrenar!

Vamos a usar chrF.

Un problema de las métricas basadas en overlap es que esperan que textos ya tokenizados, y esto dificulta la comparación entre modelos que usan  tokenizadores diferentes.

Entonces se usan librerías que aplican la misma tokenización para cualquier texto. La más usada es SacreBLEU.

In [ ]:
import evaluate

metric = evaluate.load("chrf")

In [ ]:
# Un ejemplo
predictions = [
    "volando en el azul pintado"
]
references = [
    [
        "volar en el azul pintado de azul"
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 62.37172019522234, 'char_order': 6, 'word_order': 0, 'beta': 2}

### Fine-tuning

Entrenamos con un loop custom en lugar del wrapper Trainer de Hugging Face.

In [ ]:
# Necesitamos un collator que:
#   * Haga padding dinámicamente (a la longitud máxima), tanto en tokens source como target
#   * reemplace con -100 los labels de padding para ignorarlos en la loss

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# DataLoaders
from torch.utils.data import DataLoader

train_device_bsz = 16 # el size efectivo es esto x nro de GPUs
dev_device_bsz = 16

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=train_device_bsz,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=dev_device_bsz
)

In [ ]:
# Modelo
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# Optimizer
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5) # para FT solemos usar LR baja

Usamos accelerate para entrenar en muchas GPUs en paralelo. Otra librería muy usada para hacer esto es pytorch-lightning.

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
# Chequeamos el nro de GPUs:
torch.cuda.device_count() # una sola :(

1

In [ ]:
# Usamos un scheduling de la LR desde lr hasta 0
#  (en general ir reduciendo la LR ayuda a la convergencia)
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
# procesamiento de las salidas para hacer la evaluación
import numpy as np

def postprocess(predictions, labels):
    """predictions: lista de tokens IDs generados (hypothesis)
    labels: lista de tokens IDs correctos (reference)
    """
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # decoding solo de los labels validos
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # limpiamos:
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
# el loop!
from tqdm.auto import tqdm
import torch

output_dir = "marian-ft-es-it_PRUEBA"

train_pbar = tqdm(range(num_training_steps), unit="batch", desc="Training")

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch) # forward
        loss = outputs.loss # loss
        accelerator.backward(loss) # backward
        optimizer.step() # ajuste
        lr_scheduler.step() # actualizamos LR
        optimizer.zero_grad() # gradientes a cero
        train_pbar.update(1)

    # Evaluación (solo al final de cada epoch)
    # Ojo que tambien estamos evaluando el método de generacion!!!
    model.eval()
    for batch in tqdm(eval_dataloader, unit="batch", desc="Evaluation"):
        with torch.inference_mode():
            # Tenemos que hacer unwrap porque el modelo preparado por accelerate no tiene metodo generate!
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128, # Ojo con el max_length!
            )
        labels = batch["labels"]

        # En cada proceso lanzado por accelerate se puede haber hecho padding de inputs y labels con distinto largo
        # Tenemos que llevarlos al mismo largo antes de juntarlos en un solo tensor (gather)
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)
        # Juntamos los datos de los distintos devices
        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)
        # Mandamos labels y predicciones a la métrica
        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, metric: {results['score']:.2f}")

    # Save model
    # NOTE idealmente nos gustaría guardar el mejor, no el último
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

Training:   0%|          | 0/375 [00:00<?, ?batch/s]

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Evaluation:   0%|          | 0/125 [00:00<?, ?batch/s]

epoch 0, metric: 61.72


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59030]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Evaluation:   0%|          | 0/125 [00:00<?, ?batch/s]

epoch 1, metric: 61.57


Evaluation:   0%|          | 0/125 [00:00<?, ?batch/s]

epoch 2, metric: 61.57


### Inferencia

Podemos hacer inferencia con un pipeline para ahorrarnos todos los pasos de inferencia intermedios.

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model=output_dir)

Device set to use cuda:0


In [ ]:
translator("la ilustración y fomento de las letras es la llave maestra que abre las puertas de la abundancia y hace felices a los pueblos")

[{'translation_text': "l'illustrazione e la promozione delle lettere è la chiave maestra che apre le porte dell'abbondanza e rende felici i popoli"}]

**PREGUNTA**: ¿cómo podemos evaluar si el modelo fine-tuneado es mejor que el que usamos de base?

## Referencias

* https://github.com/huggingface/transformers/tree/main/examples/pytorch
* https://huggingface.co/learn/llm-course/en/chapter7/4?fw=pt
* https://huggingface.co/docs/transformers/en/model_doc/encoder-decoder
* https://huggingface.co/blog/encoder-decoder
* https://huggingface.co/docs/transformers/en/model_doc/marian#marianmt
* https://huggingface.co/docs/transformers/main/en/generation_strategies
